## FY23 FTA Bus and Low- and No-Emission Grant Awards Analysis

<b>GH issue:</b> 
* Research Request - Bus Procurement Costs & Awards #897

<b>Data source(s):</b> 
1. https://www.transit.dot.gov/funding/grants/fy23-fta-bus-and-low-and-no-emission-grant-awards
2. https://storymaps.arcgis.com/stories/022abf31cedd438b808ec2b827b6faff

<b>Definitions:</b>  
* <u>Grants for Buses and Bus Facilities Program:</u>
    * 49 U.S.C. 5339(b)) makes federal resources available to states and direct recipients to replace, rehabilitate and purchase buses and related equipment and to construct bus-related facilities, including technological changes or innovations to modify low or no emission vehicles or facilities. Funding is provided through formula allocations and competitive grants. 
<br><br>
* <u>Low or No Emission Vehicle Program:</u>
    * 5339(c) provides funding to state and local governmental authorities for the purchase or lease of zero-emission and low-emission transit buses as well as acquisition, construction, and leasing of required supporting facilities.


In [1]:
# import shared_utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm

# set_option to increase max rows displayed to 200, to see entire df in 1 go/
pd.set_option("display.max_rows", 300)

## Reading in raw data from gcs

In [2]:
df = pd.read_csv(
    "gs://calitp-analytics-data/data-analyses/bus_procurement_cost/data-analyses_bus_procurement_cost_fta_press_release_data_csv.csv"
)

## Data Cleaning
1. snake-case column names
2. remove currency formatting from funding column (with $ and , )
3. seperate text from # of bus col (split at '(')
    a. trim spaces in new col
    b. get rid of () characters in new col
4. trim spaces in other columns
5. exnamine column values and replace/update as needed
6. create new columns for bus size type and prop type


### Dataframe cleaning

In [3]:
# fucntions to clean up dataframe and df columns
def snake_case(df):
    df.columns=df.columns.str.lower()
    df.columns=df.columns.str.replace(" ", "_")
    df.columns=df.columns.str.strip()

def fund_cleaner(df,column):
    df[column]= df[column].str.replace("$", "")
    df[column]= df[column].str.replace(",", "")
    df[column]= df[column].str.strip()



In [4]:
#snake case function to Df
snake_case(df)

### Column Cleaning

#### propulsion_type

In [5]:
#rename col to propulsion category
df=df.rename(columns={'propulsion_type':'propulsion_category'})

In [6]:
# make values in prop_cat col lower case and remove spaces
df["propulsion_category"] = df["propulsion_category"].str.lower()
df["propulsion_category"] = df["propulsion_category"].str.replace(" ", "")

In [ ]:
df.head(3)

#### funding

In [8]:
fund_cleaner(df,'funding')

/tmp/ipykernel_465/4154908731.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[column]= df[column].str.replace("$", "")


In [9]:
df["funding"] = df["funding"].astype("int64")

In [ ]:
df.columns

#### approx_#_of_buses to bus_count and prop_type

In [10]:
# test of removing the spaces first in # of bus colum, THEN split by (
df["approx_#_of_buses"] = df["approx_#_of_buses"].str.replace(" ", "")

In [11]:
# spliting the # of buses column into 2, using the ( char as the delimiter
df[["bus_count", "prop_type"]] = df["approx_#_of_buses"].str.split(pat="(", n=1, expand=True)

In [ ]:
df.head(3)

#### bus_count

In [12]:
# function to find the row index of a specific value and column in a dataframe
def find_loc(data, col, val):
    x = data.loc[data[col] == val].index[0]
    return x

In [13]:
loc1 = find_loc(df, "bus_count", "56estimated-cutawayvans")
loc2 = find_loc(df, "bus_count", "12batteryelectric")

In [14]:
display(loc1, loc2)

58

32

In [15]:
# editing the values of the bus count col at specific location
# syntax, look at ## index, look at XX column
df.loc[58, "bus_count"] = 56
df.loc[32, "bus_count"] = 12

In [16]:
# updating values again for bus_desc. same location
df.loc[58, "prop_type"] = "estimated-cutaway vans (PM- award will not fund 68 buses)"
df.loc[32, "prop_type"] = "battery electric"

In [17]:
# bus count for row 12 needs to be adjusted to 31 instead of 15
df.loc[12, "bus_count"] = 31

In [ ]:
# confirming the change
df.loc[12]

#### project_type

In [19]:
# using str.lower() on project type
df["project_type"] = df["project_type"].str.lower()
# using str.lower() on project type
df["project_type"] = df["project_type"].str.replace(" ", "")

In [20]:
# some values still need to get adjusted. will use a short dictionary to fix
new_type = {
    "\tbus/facility": "bus/facility",
    "bus/facilitiy": "bus/facility",
    "facilities": "facility",
}

In [21]:
# using replace() with the dictionary to replace keys in project type col
# syntax df.replace({'bus_desc': new_dict}, inplace=True)
df.replace({"project_type": new_type}, inplace=True)

In [ ]:
df.project_type.unique()

#### `prop_type`

In [22]:
# clearning the bus desc/prop_type col.
# removing the )
df["prop_type"] = df["prop_type"].str.replace(")", "")

/tmp/ipykernel_465/2225036280.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["prop_type"] = df["prop_type"].str.replace(")", "")


In [ ]:
df["prop_type"].unique()

In [24]:
# stripping the values in the bus desc col
df["prop_type"] = df["prop_type"].str.strip()

In [26]:
# creating a dictionary to add spaces back to the values
spaces = {
    "beb": "BEB",
    "estimated-CNGbuses": "estimated-CNG buses",
    "cngbuses": "CNG buses",
    "BEBs": "BEB",
    "Electric\n16(Hybrid": "15 electic, 16 hybrid",
    "FuelCellElectric": "fuel cell electric",
    "FuelCell": "fuel cell",
    "lowemissionCNG": "low emission CNG",
    "cng": "CNG",
    "BEBsparatransitbuses": "BEBs paratransit buses",
    "hybridelectric": "hybrid electric",
    "zeroemissionbuses": "zero emission buses",
    "dieselelectrichybrids": "diesel electric hybrids",
    "hydrogenfuelcell": "hydrogen fuel cell",
    "2BEBsand4HydrogenFuelCellBuses": "2 BEBs and 4 hydrogen fuel cell buses",
    "4fuelcell/3CNG": "4 fuel cell / 3 CNG",
    "hybridelectricbuses": "hybrid electric buses",
    "CNGfueled": "CNG fueled",
    "zeroemissionelectric": "zero emission electric",
    "hybridelectrics": "hybrid electrics",
    "dieselandgas": "diesel and gas",
    "diesel-electrichybrids": "diesel-electric hybrids",
    "propanebuses": "propane buses",
    "1:CNGbus;2cutawayCNGbuses": "1:CNGbus ;2 cutaway CNG buses",
    "zeroemission": "zero emission",
    "propanedpoweredvehicles": "propaned powered vehicles",
}

In [27]:
# using new dictionary to replace values in the bus desc col
df.replace({"prop_type": spaces}, inplace=True)

In [29]:
list(df['prop_type'].sort_values().unique())

['15 electic, 16 hybrid',
 '1:CNGbus ;2 cutaway CNG buses',
 '2 BEBs and 4 hydrogen fuel cell buses',
 '4 fuel cell / 3 CNG',
 'BEB',
 'BEBs paratransit buses',
 'CNG',
 'CNG buses',
 'CNG fueled',
 'Electric',
 'FCEB',
 'battery electric',
 'diesel and gas',
 'diesel electric hybrids',
 'diesel-electric',
 'diesel-electric hybrids',
 'electric',
 'estimated-CNG buses',
 'estimated-cutaway vans (PM- award will not fund 68 buses',
 'fuel cell',
 'fuel cell electric',
 'hybrid',
 'hybrid electric',
 'hybrid electric buses',
 'hybrid electrics',
 'hydrogen fuel cell',
 'low emission CNG',
 'propane',
 'propane buses',
 'propaned powered vehicles',
 'zero emission',
 'zero emission buses',
 'zero emission electric',
 'zero-emission',
 None]

In [33]:
prop_type_dict ={'15 electic, 16 hybrid': 'mix (zero and low emission buses)',
 '1:CNGbus ;2 cutaway CNG buses': 'mix (zero and low emission buses)',
 '2 BEBs and 4 hydrogen fuel cell buses': 'mix (BEB and FCEB)',
 '4 fuel cell / 3 CNG': 'mix (zero and low emission buses)',
 'BEBs paratransit buses': "BEB",
"CNG buses": "CNG",
    "CNG fueled": "CNG",
 "Electric": "electrc (not specified)",
 "battery electric": "BEB",
 'diesel and gas': 'mix (low emission)',
 'diesel electric hybrids':'low emission (hybrid)',
 'diesel-electric':'low emission (hybrid)',
 'diesel-electric hybrids':'low emission (hybrid)',
 'electric':"electrc (not specified)",
 'estimated-CNG buses': "CNG",
 'estimated-cutaway vans (PM- award will not fund 68 buses': 'not specified',
 'fuel cell': 'FCEB',
 'fuel cell electric': 'FCEB',
 'hybrid':'low emission (hybrid)',
 'hybrid electric':'low emission (hybrid)',
 'hybrid electric buses':'low emission (hybrid)',
 'hybrid electrics':'low emission (hybrid)',
 'hydrogen fuel cell': 'FCEB',
 'low emission CNG': 'CNG',
 'propane':'low emission (propane)',
 'propane buses':'low emission (propane)',
 'propaned powered vehicles':'low emission (propane)',
 'zero emission':"zero-emission bus (not specified)",
 'zero emission buses':"zero-emission bus (not specified)",
 'zero emission electric':"zero-emission bus (not specified)",
 'zero-emission':"zero-emission bus (not specified)",
}

In [34]:
# repalcing values in prop type with prop type dictionary
df.replace({"prop_type": prop_type_dict}, inplace=True)

In [36]:
#check work
list(df.prop_type.sort_values().unique())

['BEB',
 'CNG',
 'FCEB',
 'electrc (not specified)',
 'low emission (hybrid)',
 'low emission (propane)',
 'mix (BEB and FCEB)',
 'mix (low emission)',
 'mix (zero and low emission buses)',
 'not specified',
 'zero-emission bus (not specified)',
 None]

In [ ]:
df.head()

### Need new column for `bus size type` via list and function
cutaway, 40ft etc

In [ ]:
list(df.columns)

In [37]:
bus_size = [
    "standard",
    "40 foot",
    "40-foot",
    "40ft",
    "articulated",
    "cutaway",
]

In [38]:
# Function to match keywords
def find_bus_size_type(description):
    for keyword in bus_size:
        if keyword in description.lower():
            return keyword
    return "not specified"

In [39]:
# new column called bus size type based on description column
df["bus_size_type"] = df["description"].apply(find_bus_size_type)

In [ ]:
#check work
display(
    df.head(3),
    df.bus_size_type.unique(),
    df.shape
)

## Exporting cleaned data to GCS

In [40]:
# saving to GCS as csv
df.to_csv(
    "gs://calitp-analytics-data/data-analyses/bus_procurement_cost/fta_bus_cost_clean.csv"
)

## Reading in cleaned data from GCS

In [41]:
bus_cost = pd.read_csv(
    "gs://calitp-analytics-data/data-analyses/bus_procurement_cost/fta_bus_cost_clean.csv"
)

In [ ]:
# drop unnessary columns
bus_cost = bus_cost.drop(["Unnamed: 0", "congressional_districts"], axis=1)

In [42]:
# confirming cleaned data shows as expected.
display(bus_cost.shape, type(bus_cost), bus_cost.columns)

(130, 16)

pandas.core.frame.DataFrame

Index(['Unnamed: 0', 'state', 'project_sponsor', 'project_title',
       'description', 'funding', 'approx_#_of_buses', 'project_type',
       'propulsion_category', 'area_served', 'congressional_districts',
       'fta_region', 'bus/low-no_program', 'bus_count', 'prop_type',
       'bus_size_type'],
      dtype='object')

In [44]:
bus_cost['prop_type'].sort_values(ascending=True).unique()

array(['BEB', 'CNG', 'FCEB', 'electrc (not specified)',
       'low emission (hybrid)', 'low emission (propane)',
       'mix (BEB and FCEB)', 'mix (low emission)',
       'mix (zero and low emission buses)', 'not specified',
       'zero-emission bus (not specified)', nan], dtype=object)

## DEPRECATED - Data Analysis
actual data analysis and summary stats exist in the `cost_per_bus_analysis.ipynb` notebook

### Cost per Bus, per Transit Agency dataframe

In [ ]:
only_bus = bus_cost[bus_cost["bus_count"] > 0]
only_bus.head()

In [ ]:
cost_per_bus = (
    only_bus.groupby("project_sponsor")
    .agg({"funding": "sum", "bus_count": "sum"})
    .reset_index()
)

In [ ]:
cost_per_bus["cost_per_bus"] = (
    cost_per_bus["funding"] / cost_per_bus["bus_count"]
).astype("int64")

In [ ]:
cost_per_bus.dtypes

In [ ]:
cost_per_bus

In [ ]:
## export cost_per_bus df to gcs
cost_per_bus.to_csv(
    "gs://calitp-analytics-data/data-analyses/bus_procurement_cost/fta_cost_per_bus.csv"
)

### Cost per bus, stats analysis

In [ ]:
# read in fta cost per bus csv
cost_per_bus = pd.read_csv(
    "gs://calitp-analytics-data/data-analyses/bus_procurement_cost/fta_cost_per_bus.csv"
)

In [ ]:
display(cost_per_bus.shape, cost_per_bus.head())

### Initial Summary Stats

### Summary Stats

In [ ]:
# top level alanysis

bus_cost.agg({"project_title": "count", "funding": "sum", "bus_count": "sum"})

In [ ]:
# start of agg. by project_type

bus_cost.groupby("project_type").agg(
    {"project_type": "count", "funding": "sum", "bus_count": "sum"}
)

In [ ]:
# agg by program

bus_cost.groupby("bus/low-no_program").agg(
    {"project_type": "count", "funding": "sum", "bus_count": "sum"}
)

In [ ]:
# agg by state, by funding
bus_cost.groupby("state").agg(
    {"project_type": "count", "funding": "sum", "bus_count": "sum"}
).sort_values(by="funding", ascending=False)

### Projects with bus purchases

In [ ]:
# df of only projects with a bus count
only_bus = bus_cost[bus_cost["bus_count"] > 0]

In [ ]:
display(only_bus.shape, only_bus.columns)

In [ ]:
# agg by propulsion type
only_bus["propulsion_type"].value_counts()

In [ ]:
only_bus.project_type.value_counts()

In [ ]:
# of the rows with bus_count >1, what are the project types?
bus_agg = only_bus.groupby("project_type").agg(
    {"project_type": "count", "funding": "sum", "bus_count": "sum"}
)

In [ ]:
# new column that calculates `cost per bus`
bus_agg["cost_per_bus"] = (bus_agg["funding"] / bus_agg["bus_count"]).astype("int64")

In [ ]:
bus_agg

### Projects with no buses

In [ ]:
no_bus = bus_cost[bus_cost["bus_count"] < 1]

In [ ]:
no_bus["project_type"].value_counts()

### Overall Summary

In [ ]:
project_count = bus_cost.project_title.count()
fund_sum = bus_cost.funding.sum()
bus_count_sum = bus_cost.bus_count.sum()
overall_cost_per_bus = (fund_sum) / (bus_count_sum)
bus_program_count = bus_cost["bus/low-no_program"].value_counts()

projects_with_bus = only_bus.project_title.count()
projects_with_bus_funds = only_bus.funding.sum()
cost_per_bus = (only_bus.funding.sum()) / (bus_count_sum)

In [ ]:
summary = f"""
Top Level observation:
- {project_count} projects awarded
- ${fund_sum:,.2f} dollars awarded
- {bus_count_sum} buses to be purchased
- ${overall_cost_per_bus:,.2f} overall cost per bus

Projects have some mix of buses, facilities and equipment. Making it difficult to disaggregate actual bus cost.

Of the {project_count} projects awarded, {projects_with_bus} projects inlcuded buses. The remainder were facilities, chargers and equipment

Projects with buses purchases:
- {projects_with_bus} projects
- ${projects_with_bus_funds:,.2f} awarded to purchases buses
- ${cost_per_bus:,.2f} cost per bus
"""

In [ ]:
print(summary)

In [ ]:
# Assuming your DataFrame is named df
cost_per_bus_values = cost_per_bus["cost_per_bus"]

# Calculate mean and standard deviation
mean_value = cost_per_bus_values.mean()
std_deviation = cost_per_bus_values.std()

# Plot histogram
plt.hist(cost_per_bus_values, bins=30, color="skyblue", edgecolor="black", alpha=0.7)

# Add vertical lines for mean and standard deviation
plt.axvline(mean_value, color="red", linestyle="dashed", linewidth=2, label="Mean")
plt.axvline(
    mean_value + std_deviation,
    color="green",
    linestyle="dashed",
    linewidth=2,
    label="Mean + 1 Std Dev",
)
plt.axvline(
    mean_value - std_deviation,
    color="green",
    linestyle="dashed",
    linewidth=2,
    label="Mean - 1 Std Dev",
)

# Set labels and title
plt.xlabel("cost_per_bus")
plt.ylabel("Frequency")
plt.title("Histogram of cost_per_bus with Mean and Std Dev Lines")
plt.legend()

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd

# Assuming your DataFrame is named df
cost_per_bus_values = cost_per_bus["cost_per_bus"]

# Calculate mean and standard deviation
mean_value = cost_per_bus_values.mean()
std_deviation = cost_per_bus_values.std()

# Plot histogram
plt.hist(cost_per_bus_values, bins=20, color="skyblue", edgecolor="black", alpha=0.7)

# Add vertical lines for mean and standard deviation
plt.axvline(mean_value, color="red", linestyle="dashed", linewidth=2, label="Mean")
plt.axvline(
    mean_value + std_deviation,
    color="green",
    linestyle="dashed",
    linewidth=2,
    label="Mean + 1 Std Dev",
)
plt.axvline(
    mean_value - std_deviation,
    color="green",
    linestyle="dashed",
    linewidth=2,
    label="Mean - 1 Std Dev",
)

# Set labels and title
plt.xlabel("Cost per Bus (USD)")
plt.ylabel("Frequency")
plt.title("Histogram of Cost per Bus with Mean and Std Dev Lines")
plt.legend()

# Format x-axis ticks as USD
plt.gca().xaxis.set_major_formatter(mticker.StrMethodFormatter("${x:,.0f}"))

# Show the plot
plt.show()